## AIDevs2 tasks from newest at the top to oldest at the bottom ,updated daily, to be cleaned ;)

In [30]:
import requests
from utilities.config import AI_DEVS_SERVER, AI_DEVS_API_KEY, OPEN_AI_API_KEY
import openai

#To make work, create .env file
BASE_URL = AI_DEVS_SERVER
API_KEY = AI_DEVS_API_KEY
openai.api_key  = OPEN_AI_API_KEY

In [31]:
API_KEY

''

#!pip install --upgrade openai


In [8]:
def get_token(task_name):
    url = f"{BASE_URL}/token/{task_name}"
    payload = {
        "apikey": API_KEY
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        return response.json()  # Return the entire JSON response
    else:
        print(f"Error getting token: {response.text}")
        return None

def get_task(token):
    url = f"{BASE_URL}/task/{token}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()  # Return the entire JSON response
    else:
        print(f"Error getting task: {response.text}")
        return None

def submit_answer(token, answer):
    url = f"{BASE_URL}/answer/{token}"
    payload = {
        "answer": answer
    }
    response = requests.post(url, json=payload)
    return response.json()  # Return the entire JSON response

def get_completion(prompt, model="gpt-4", max_tokens=300, temperature=0.3): # or model="gpt-3.5-turbo"
    messages = [{"role": "user", "content": prompt}]
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return response.choices[0].message["content"]
    except openai.error.OpenAIError as e:
        print(f"OpenAI error: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


# liar

Jest to mechanizm, który mówi nie na temat w 1/3 przypadków. Twoje zadanie polega na tym, aby do endpointa /task/ wysłać swoje pytanie w języku angielskim (dowolne, np “What is capital of Poland?’) w polu o nazwie ‘question’ (metoda POST, jako zwykłe pole formularza, NIE JSON). System API odpowie na to pytanie (w polu ‘answer’) lub zacznie opowiadać o czymś zupełnie innym, zmieniając temat. Twoim zadaniem jest napisanie systemu filtrującego (Guardrails), który określi (YES/NO), czy odpowiedź jest na temat. Następnie swój werdykt zwróć do systemu sprawdzającego jako pojedyncze słowo YES/NO. Jeśli pobierzesz treść zadania przez API bez wysyłania żadnych dodatkowych parametrów, otrzymasz komplet podpowiedzi. Skąd wiedzieć, czy odpowiedź jest ‘na temat’? Jeśli Twoje pytanie dotyczyło stolicy Polski, a w odpowiedzi otrzymasz spis zabytków w Rzymie, to odpowiedź, którą należy wysłać do API to NO.

In [9]:
task_name = "liar"

In [10]:
def ask_question(token, question):
    url = f"{BASE_URL}/task/{token}"
    payload = {
        "question": question
    }
    response = requests.post(url, data=payload)  # Using data instead of json
    if response.status_code == 200:
        return response.json()  # Return the entire JSON response
    else:
        print(f"Error asking question: {response.text}")
        return None

In [11]:
#question = "What the moon?"
question = get_completion("respond with random question", model="gpt-3.5-turbo", max_tokens=100, temperature=0.8)

question

OpenAI error: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.


In [12]:
token = get_token(task_name)['token']

answer_to_question = ask_question(token, question)['answer']

print(question + " : " + answer_to_question)

Error getting token: {
    "code": -3,
    "msg": "wrong API key"
}


TypeError: 'NoneType' object is not subscriptable

Guardrails

In [13]:
prompt = f"""
Given the question '{question}' and the answer '{answer_to_question}', is the answer relevant? Respond with YES or NO.
"""

response = get_completion(prompt, model="gpt-3.5-turbo", max_tokens=100, temperature=0.2)

response.strip()  # Stripping to ensure there's no extra whitespace

NameError: name 'answer_to_question' is not defined

In [14]:
submit_answer(token, response)

NameError: name 'token' is not defined

# blogger

Napisz wpis na bloga (w języku polskim) na temat przyrządzania pizzy Margherity. Zadanie w API nazywa się ”blogger”. Jako wejście otrzymasz spis 4 rozdziałów, które muszą pojawić się we wpisie. Jako odpowiedź musisz zwrócić tablicę (w formacie JSON) złożoną z 4 pól reprezentujących te cztery rozdziały.

In [15]:
task_name = "blogger"

In [16]:
token = get_token(task_name)['token']
task = get_task(token)['blog']

print(task)

Error getting token: {
    "code": -3,
    "msg": "wrong API key"
}


TypeError: 'NoneType' object is not subscriptable

Writing a Blog Post - Best Solution - in One Prompt and Dividing Afterward

In [ ]:
prompt = f"""
Write a Polish blog post about the preparation of Margherita pizza. Please elaborate on each of the following chapters, and separate each chapter with the keyword '***':
{'***'.join(task)}
"""
response = get_completion(prompt, model="gpt-3.5-turbo", max_tokens=1000, temperature=0.3)

# Post-process the response into a list of strings, where each string is a chapter
blog_post = [chapter.strip() for chapter in response.split('***')]

print(blog_post)

['Wstęp: kilka słów na temat historii pizzy\n\nPizza jest jednym z najbardziej znanych i uwielbianych dań na całym świecie. Jej historia sięga starożytności, a dokładniej do starożytnego Rzymu. Początkowo była to prosta płaska bułka, którą podawano z różnymi dodatkami. Jednak to we Włoszech pizza zyskała swoją popularność i stała się symbolem tego kraju. Dziś pizza jest dostępna w wielu różnych wariantach, ale jednym z najbardziej klasycznych i uwielbianych jest Margherita.', 'Niezbędne składniki na pizzę\n\nAby przygotować pyszną Margheritę, potrzebujemy kilku podstawowych składników. Przede wszystkim musimy mieć dobrej jakości mąkę, najlepiej typu 00, która jest najbardziej odpowiednia do pizzy. Potrzebujemy również drożdży, które dodamy do mąki w celu wyrośnięcia ciasta. Kolejnym ważnym składnikiem jest sos pomidorowy. Możemy go przygotować samodzielnie, gotując pomidory i dodając przyprawy, lub skorzystać z gotowego sosu pomidorowego. Oprócz tego potrzebujemy mozzarelli, która jest

In [ ]:
result = submit_answer(token, blog_post)

print(result)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


# moderation

Zastosuj wiedzę na temat działania modułu do moderacji treści i rozwiąż zadanie o nazwie “moderation” z użyciem naszego API do sprawdzania rozwiązań. Zadanie polega na odebraniu tablicy zdań (4 sztuki), a następnie zwróceniu tablicy z informacją, które zdania nie przeszły moderacji. Jeśli moderacji nie przeszło pierwsze i ostatnie zdanie, to odpowiedź powinna brzmieć [1,0,0,1]. Pamiętaj, aby w polu ‘answer’ zwrócić tablicę w JSON, a nie czystego stringa.

In [17]:
task_name = "moderation"

In [18]:
token = get_token(task_name)['token']
task = get_task(token)['input']

print(task)

Error getting token: {
    "code": -3,
    "msg": "wrong API key"
}


TypeError: 'NoneType' object is not subscriptable

Moderation

In [19]:
# Initialize an empty list to hold the moderation results
moderation_results = []

# Iterate through each item in the zadanie list
for item in task:
    # Send the item to the Moderation endpoint
    response = openai.Moderation.create(input=item)
    
    # Access the 'flagged' value from the response
    flagged = response['results'][0]['flagged']
    
    # Append 1 if flagged, else 0, to the moderation_results list
    moderation_results.append(1 if flagged else 0)

NameError: name 'task' is not defined

In [20]:
moderation_results


[]

In [21]:
result = submit_answer(token, moderation_results)

print(result)

NameError: name 'token' is not defined

# helloapi

Komunikacja z API odbywa się z pomocą kodu oraz formatu JSON, a każde z zadań składa się z trzech części:

autoryzacji

pobierania danych wejściowych (string lub tablica obiektów)

odesłania odpowiedzi (właściwość answer)

In [22]:
task_name = "helloapi"

In [23]:
token = get_token(task_name)['token']
cookie = get_task(token)['cookie']
result = submit_answer(token, cookie)

print(result)


Error getting token: {
    "code": -3,
    "msg": "wrong API key"
}


TypeError: 'NoneType' object is not subscriptable